# ДПО ВШЭ

## Современный анализ данных, глубокое обучение и приложения

## Задание на тему: "Соревнования по анализу данных"

В этом задании мы будем решать соревнование по определению лояльности пользователей сети [Kaggle Elo Merchant Category Recommendation](https://www.kaggle.com/c/elo-merchant-category-recommendation).

Сначала мы будем действовать самостоятельно (загрузим данные, посмотрим на них, прикинем, что можно сделать, подготовим простое решение и сравним его с локальной валидацией). Далее перейдём к особенностям решения соревнований на платформе (изучение публичных решений и форума).

Для начала необходимо загрузить все данные и расположить их рядом с этим ноутбуком.

In [1]:
%pylab inline
import pandas as pd

Populating the interactive namespace from numpy and matplotlib


/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


## Часть 1. Своё решение

### Загрузка данных

In [2]:
data_dir = 'elo-merchant-category-recommendation/'

In [3]:
train = pd.read_csv(data_dir + 'train.csv')
test = pd.read_csv(data_dir + 'test.csv')

In [4]:
train.head()

,first_active_month,card_id,feature_1,feature_2,feature_3,target
0,2017-06,C_ID_92a2005557,5,2,1,-0.820283
1,2017-01,C_ID_3d0044924f,4,1,0,0.392913
2,2016-08,C_ID_d639edf6cd,2,2,0,0.688056
3,2017-09,C_ID_186d6a6901,4,3,0,0.142495
4,2017-11,C_ID_cdbd2c0db2,1,3,0,-0.159749


In [5]:
transactions = pd.read_csv(data_dir + 'historical_transactions.csv')

In [6]:
transactions.head()

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
0,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_e020e9b302,-8,-0.703331,2017-06-25 15:33:07,1.0,16,37
1,Y,C_ID_4e6213e9bc,88,N,0,A,367,M_ID_86ec983688,-7,-0.733128,2017-07-15 12:10:45,1.0,16,16
2,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_979ed661fc,-6,-0.720386,2017-08-09 22:04:29,1.0,16,37
3,Y,C_ID_4e6213e9bc,88,N,0,A,560,M_ID_e6d5ae8ea6,-5,-0.735352,2017-09-02 10:06:26,1.0,16,34
4,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_e020e9b302,-11,-0.722865,2017-03-10 01:14:19,1.0,16,37


In [ ]:
# new_merchant_transactions = pd.read_csv(
#     data_dir + 'new_merchant_transactions.csv')

In [7]:
merchants = pd.read_csv(data_dir + 'merchants.csv')

In [8]:
merchants.head()

,merchant_id,merchant_group_id,merchant_category_id,subsector_id,numerical_1,numerical_2,category_1,most_recent_sales_range,most_recent_purchases_range,avg_sales_lag3,...,avg_sales_lag6,avg_purchases_lag6,active_months_lag6,avg_sales_lag12,avg_purchases_lag12,active_months_lag12,category_4,city_id,state_id,category_2
0,M_ID_838061e48c,8353,792,9,-0.057471,-0.057471,N,E,E,-0.40,...,-2.25,18.666667,6,-2.32,13.916667,12,N,242,9,1.0
1,M_ID_9339d880ad,3184,840,20,-0.057471,-0.057471,N,E,E,-0.72,...,-0.74,1.291667,6,-0.57,1.687500,12,N,22,16,1.0
2,M_ID_e726bbae1e,447,690,1,-0.057471,-0.057471,N,E,E,-82.13,...,-82.13,260.000000,2,-82.13,260.000000,2,N,-1,5,5.0
3,M_ID_a70e9c5f81,5026,792,9,-0.057471,-0.057471,Y,E,E,NaN,...,NaN,4.666667,6,NaN,3.833333,12,Y,-1,-1,NaN
4,M_ID_64456c37ce,2228,222,21,-0.057471,-0.057471,Y,E,E,NaN,...,NaN,0.361111,6,NaN,0.347222,12,Y,-1,-1,NaN


In [ ]:
# your code

**Ответьте на вопросы:** (по 1-2 предложениям на вопрос)

1. Какая информация доступна нам?
1. Чем характеризуется лояльность покупателя? (по собственному опыту)
1. Какие признаки в первую очередь можно было бы построить на текущих данных?
1. Какая метрика у данной задачи?

**Ответы:**
...

### Первичная аналитика

**Ответьте на следующие вопросы.** Для ответа вам необходимо произвести некоторую аналитику. К каждому ответу нужно оставить поясняющую ваш ответ иллюстрацию или краткое рассуждение.

1. Посмотрите распределение целевой переменной. Есть ли что-то интересное? Как это согласуется с метрикой в задаче?
1. Сравните обучающую и тестовую выборки. Как, во-вашему, разбиты данные на обучение и тест? Какую валидацию можно было бы использовать?

**Ответы:** ...

In [ ]:
# your code

### Бейзлайн-решение

Готовить бейзлайн решение будем вместе с валидацией. Возьмём несколько исходных признаков (не забудьте про их типы!) и добавим несколько самых простых на основе других таблиц. Не забудем про даты и признаки на их основе. 

**Подготовьте около 10 признаков для обучающей и тестовой выборки**.

In [ ]:
# your code

**Попробуйте обучить модель и оценить качество локально** (возьмите тот тип валидации, который считаете нужным). Если ваш тип валидации подразумевает случайность, зафиксируйте параметры случайности.

Попробуйте и градиентный бустинг, и гребневую регрессию из удобных вам библиотек.

In [ ]:
# your code

Проверить, что модель выучивает "что-то" можно, сравнив её с самым простым и тупым решением — константным предсказанием. Для простоты можно не делать это честно через используемый способ валидации.

**Ответьте:**
1. Какая константа будет оптимальной для нашего функционала? (Если не помните, то найдите минимум функции потерь через дифференцирование функции).
2. Как отличается качество у константного решения и двух обученных моделей?

**Ответы:**

In [ ]:
# your code

Обучите лучшую из моделей на всей обучающей выборке, сделайте предсказание на тестовой. Сохраните ваши предсказания и отправьте их на платформу. Какое качество получилось на лидерборде? Как это соотносится с локальным качеством?

**Ответы:**

In [ ]:
# your code

Попробуйте немного изменить параметры модели или добавить новых признаков. Оцените локальное качество и сравните его с лидербордом. Что у вас получилось?

**Ответ:**

In [ ]:
# your code

Возможно, вы замечали что-то интересное при изучении распределения целевой переменной. Как это могло отразиться на вашем решении? Что с этим можно сделать?

**Ответы:**

In [ ]:
# your code

Метрика этой задачи допускает проверку среднего значения на тестовой части выборки. Попробуйте вычислить среднее значение на публичной части тестовой выборки (через 2 сабмита). 

Для простоты можно проверить (через 3 сабмита), что, например, среднее значение на тесте больше или меньше среднего значения на обучающей выборке на некоторую небольшую величину (выбирается самостоятельно).

Отличается ли значение от среднее значение на тестовой выборке от среднего значения на обучающей выборке?

**Ответ:**

In [ ]:
# your code

Если вы не кодировали категориальные признаки через mean-target кодирование ранее, то попробуйте сделать это сейчас. Для простоты не требуется воспроизводить схему с двойной кросс валидацией. Получилось ли улучшить качество?

**Ответ:**

## Часть 2. Работа с платформой 

Как только мы разобрались с задачей и попробовали что-то порешать (как это могло быть и в реальной задаче), попробуем воспользоваться преимуществом решения задач на платформе.

### Kernels

Начнём с kernels (публичных решений).

Перейдите на страницу с публичными решениями, отсортируйте их по количество голосов (most votes). Просмотрите один из первых kernel EDA (изучение данных).

1. Какие интересные визуализации сделаны? (технически или идейно сложные)
1. Какие зависимости в данных удалось найти автору?
1. В каком из источников (таблиц) с информацией больше всего полезной информации?

**Ответы:**

Теперь просмотрите несколько решений (не EDA).

1. Какие методы (алгоритмы) и техники используют авторы?
1. Каких результатов им удаётся достичь? (качество и примерное место на лидерборде)
1. Что интересного в этих решениях можно увидеть? На чём они основаны? (работа с признаками / тюнинг моделей / техники ансамблирования или постпроцессинга)
1. Что плохого в этих решениях вы увидели?

**Ответы:**

Теперь откройте лидерборд и отматывайте вниз до тех пор, пока в графе kernel не увидите название решениея. Это лучшее на данный момент публичное решение. Откройте его и просмотрите.

На чём оно основано? Что интересного в нём есть?

**Ответы:**

Попробуйте сделать fork этого решения и изменить в нём что-либо (что по-вашему не так). Запустите решение прямо на kaggle.com. Какое качество у вас получилось?

**Ответ:**

__Сделайте ваш fork публичным и вставьте ссылку на него ниже:__

__Не забудьте написать здесь свой никнейм и лучшее качество в лидерборде:__

### Форум

Откройте вкладку с форумом (discussions). Отсортируйте темы по количеству голосов (most votes). Просмотрите темы форума, просмотрите несколько интересных по названию тем подробнее. 

Что интересного вы нашли? Что обсуждают участники на форуме?

**Ответы:**